<a href="https://colab.research.google.com/github/UncleSamTech/301Lab1WalkThrough/blob/main/bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score,confusion_matrix, classification_report
import pickle
import time
from sklearn.utils.class_weight import compute_class_weight
from random import sample
import seaborn as sns
from concurrent.futures import ProcessPoolExecutor
import heapq
import cProfile
import pstats

In [12]:
def predict_token_score(context, token, tokenz, model, maxlen):
        # Early check for out-of-vocabulary token
        LOW_SCORE =  -1
        token_index = tokenz.word_index.get(token, LOW_SCORE)
        if token_index == LOW_SCORE:
            return LOW_SCORE  # Assign low score for out-of-vocabulary token

        # Tokenize context
        context_sequence = tokenz.texts_to_sequences([context])[0]
        if not context_sequence or not context_sequence[0]:
            raise ValueError("Context could not be tokenized correctly.")

        # Append token index to context
        token_value = context_sequence + [token_index]

        # Ensure the input is the correct length
        # if len(token_value) < maxlen - 1:
        #     token_value = [0] * (maxlen - 1 - len(token_value)) + token_value
        # else:
        #     token_value = token_value[-(maxlen-1):]

        token_value = pad_sequences([token_value], maxlen=maxlen-1, padding='pre', truncating='pre')[0]

        # Prepare input as a list
        padded_in_seq = [token_value]

        # Model prediction
        prediction = model.predict(padded_in_seq, verbose=1)
        if len(prediction[0]) == 0:
            raise ValueError("Prediction output is empty.")
        return prediction[0][-1]  # Score of the token


In [37]:
def evaluate_bilstm_mrr_single_file(filename, maxlen, model, result_path):
        """
        Evaluate the MRR for a single file.
        Save the total reciprocal rank and total lines for each file to a text file.
        """

        ld = load_model(model,compile=False)
        #ld.summary()
        # Load the tokenizer
        with open(os.path.join(result_path, "tokenized_file_50embedtime1.pickle"), "rb") as tk:
            tokenz = pickle.load(tk)

        vocab = list(tokenz.word_index.keys())

        # Ensure result path exists
        os.makedirs(result_path, exist_ok=True)

        # # Process each file in the split folder
        # for split_file in sorted(os.listdir(split_folder)):
        #     split_file_path = os.path.join(split_folder, split_file)
        #     if not os.path.isfile(split_file_path):
        #         continue

        total_cumulative_rr = 0
        total_count = 0
        #profiler = cProfile.Profile()
        #profiler.enable()

        start_time = time.time()

            # Process each line in the file
        with open(filename, "r", encoding="utf-8") as f:
            for line in f:
                if not line.strip():
                    continue

                # Preprocess the line
                #line = line.replace("_", "UNDERSCORE").replace(">", "RIGHTANG").replace("<", "LEFTANG").lower()
                sentence_tokens = line.split(" ")
                if len(sentence_tokens) < 2:
                    continue

                context = " ".join(sentence_tokens[:-1])
                true_next_word = sentence_tokens[-1]

                # Compute scores for tokens
                heap = []
                for token in vocab:
                    context_score = predict_token_score(context, token, tokenz, ld, maxlen)
                    if len(heap) < 10:
                        heapq.heappush(heap, (context_score, token))
                    elif context_score > heap[0][0]:
                        heapq.heappushpop(heap, (context_score, token))

                heap.sort(reverse=True, key=lambda x: x[0])
                token_ranks = {t: rank + 1 for rank, (score, t) in enumerate(heap)}

                # Compute reciprocal rank
                true_next_word = true_next_word.strip()
                rank = token_ranks.get(true_next_word, 0)
                if rank:
                    current_rank = 1 / rank
                    total_cumulative_rr += current_rank
                    #print(f"processed line  context {context} with rank {current_rank} and tcr {total_cumulative_rr}")
                total_count += 1

            #profiler.disable()

            # Save profiling results to a file
            #profile_file = os.path.join(result_path, f"evalmrrvisib.prof")
            #with open(profile_file, "w") as pf:
                #stats = pstats.Stats(profiler, stream=pf)
                #stats.sort_stats('cumulative')
                #stats.print_stats()

            # Calculate total RR and lines for the file
            time_spent = time.time() - start_time
            result_file = os.path.join(result_path, f"kenlm_results_file_{filename}.txt")

        with open(result_file, "a") as rf:
            rf.write(f"File name : {filename}\n")
            rf.write(f"Total Reciprocal Rank: {total_cumulative_rr}\n")
            rf.write(f"Total Lines: {total_count}\n")
            rf.write(f"Time Spent: {time_spent:.2f} seconds\n")


In [39]:
evaluate_bilstm_mrr_single_file("/content/scratch_test_data_chunk_1.txt",33,"/content/main_bilstm_scratch_model_150embedtime_100.keras","/content/result_path/")

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_1_1/Cast:0", shape=(32,), dtype=float32). Expected shape (None, 38), but input has incompatible shape (32,)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(32,), dtype=int32)',)
  • training=False
  • mask=('None',)